In [1]:
# Provides things for use throughout the project
# Can be 'imported' by using %run project_base.ipynb

In [3]:
# Stdlib imports
import json
import os
import time

# Numpy/sklearn imports
import numpy as np
import sklearn.base as skb
import sklearn.preprocessing as skpr
import sklearn.pipeline as skpi
import sklearn.linear_model as sklm

# Tensorflow imports
import tensorflow as tf
tfer = tf.errors
tfe = tf.estimator
tfi = tf.initializers
tfla = tf.layers
tflog = tf.logging
tflo = tf.losses
tft = tf.train

# Helper import
# https://github.com/patrick-kidger/tools
import tools

# Imports for plotting
import ipympl
%matplotlib widget
import matplotlib.pyplot as plt
# Has side effects allowing for 3D plots
from mpl_toolkits.mplot3d import Axes3D

# Convenience imports for those files running this one
import collections as co
import functools as ft
import itertools as it

In [ ]:
%run project_base.ipynb

In [6]:
### Solutions to the Camassa--Holm equation
# i.e. functions to generate feature/label pairs

class _Solution:
    def __call__(self, point):
        """Evaluates the solution at the particular point."""
        raise NotImplementedError
        
    def on_grid(self, grid, extra=0):
        """Evaluates the solution on a grid of points.
        
        :[(float, float)] grid: The grid points on which to evaluate, as
            returned by either fine_grid or coarse_grid.
        :int extra: A nonnegative integer specifying that the array should be
            larger by this many entries. (To add extra data to later without the
            overheard of creating another array.)"""
        return np.array([self(grid_point) for grid_point in grid] 
                        + [0 for _ in range(extra)])

    
class Peakon(_Solution):
    """Simplest solution to the k=0 Camassa-Holm equation.
    
    Represents a soliton with a peak.
    """
    
    def __init__(self, c, **kwargs):
        """Peakons have precisely one parameter :c: defining their height and 
        speed. (We could also add an additional parameter defining their initial
        location, for consistency with TwoPeakon, but that's essentially
        unnecessary)
        """
        self.c = c
        super(Peakon, self).__init__(**kwargs)
        
    def __call__(self, point):
        t, x = point
        return self.c * np.exp(-1 * np.abs(x - self.c * t))
    

class TwoPeakon(_Solution):
    """Represents two solitons with peaks! Nonlinear effects start determining
    it location and magnitude.
    """
    # Nothing particularly clever is done to handle large inputs. And due to 
    # the exponentials involved, 'large' is really quite conservative!
    # A little testing suggests that -10 <= x1, x2 <= 30 are about the safe
    # limits. (p1 and p2 don't get fed into exponentials so they're probably
    # ok to play around with more.)
    
    def __init__(self, x1, x2, p1, p2, **kwargs):
        """TwoPeakons have essentially four parameters defining them.

        :x1: and :x2: are the initial locations of the peakons.
        :p1: and :p2: are the initial heights of the peakons. They must be
            positive. (Not zero!)
        """
        a = p1 * p2 * (1 - np.exp(x1 - x2))
        b = -p1 - p2
        discrim = np.sqrt(b ** 2 - 4 * a)
        twice_a = 2 * a
        l1 = (-b + discrim) / twice_a
        l2 = (-b - discrim) / twice_a
        
        tmp1 = np.exp(x2)
        a1 = -1 * (l2 - 1 / p2) * tmp1 / (2 * (l1 - l2))
        a2 = 0.5 * tmp1 - a1
        
        self.l1 = l1
        self.l2 = l2
        self.a1 = a1
        self.a2 = a2
        
        super(TwoPeakon, self).__init__(**kwargs)
        
    def __call__(self, point):
        t, x = point
        
        a1 = self.a1 * np.exp(t / self.l1)
        a2 = self.a2 * np.exp(t / self.l2)
        l1 = self.l1
        l2 = self.l2
        
        a1l1 = a1 * l1
        a2l2 = a2 * l2
        a1_a2 = a1 + a2
        tmp1 = a1l1 * l1 + a2l2 * l2
        tmp2 = a1l1 + a2l2
        
        # Can definitely optimise this some more in terms of factorising out the
        # t dependence
        p1 = tmp1 / (l1 * l2 * tmp2)
        p2 = a1_a2 / tmp2
        x1 = np.log(2 * (a1l1 * a2 * l1 + a2l2 * a1 * l2 - 2 * a1l1 * a2l2) / tmp1)
        x2 = np.log(2 * a1_a2)
        
        first_peakon = p1 * np.exp(-1 * np.abs(x - x1))
        second_peakon = p2 * np.exp(-1 * np.abs(x - x2))
        return first_peakon + second_peakon
    
    
class ThreePeakon(_Solution):
    """Represents three solitons with peaks! Nonlinear effects start determining
    it location and magnitude.
    """
    
    def __init__(self, x1, x2, x3, p1, p2, p3, **kwargs):
        """ThreePeakons have essentially six parameters defining them.

        :x1:, :x2:, :x3: are the initial locations of the peakons.
        :p1:, :p2:, :p3: are the initial heights of the peakons. They must be
            positive. (Not zero!)
        """
    

In [11]:
### Visualising the results of regressors

def make_fig(figsize=(10, 10)):
    """Creates a figure to start putting axes on."""
    return plt.figure(figsize=figsize)


def make_ax(fig, loc=(1, 1, 1)):
    """Creates an axis to start plotting on."""
    ax = fig.add_subplot(*loc, projection='3d')
    t_list, x_list = list(zip(*coarse_grid((0, 0))))
    ax.set_ylim(min(t_list), max(t_list))
    ax.set_xlim(min(x_list), max(x_list))
    return ax


def plot(ax, data, cg_or_fg=None, label=''):
    """Plots some data according to either a coarse or fine grid."""
    
    if cg_or_fg == 'cg':
        grid = coarse_grid((0, 0))
    elif cg_or_fg == 'fg':
        grid = fine_grid((0, 0))
    else:
        raise RuntimeError("Argument cg_or_fg must be 'cg' or 'fg'.")
    ax.scatter(*zip(*grid), data, label=label)